 ## XFOIL analysis with python UI

## init

In [1]:
import subprocess as sp
import shutil
import sys
import string
import shutil
from os.path import isfile, join
from os import listdir

In [2]:
import os
from pathlib import Path
import sched, time
def boostup(data_loc, airfoil_loc):
    p = Path(data_loc)
    #if the directory is there, delete it
    if p.is_dir():
        shutil.rmtree(data_loc)
    #when the directory is still ther, sleep
    while p.is_dir():
        time.sleep(2)
    #create da directory
    os.makedirs(data_loc)
    
    #get the list of dir in a path
    f = Path(airfoil_loc)
    #if the directory is there, delete it
    if f.is_dir():
        shutil.rmtree(airfoil_loc)
    #when the directory is still ther, sleep
    while f.is_dir():
        time.sleep(2)
    #create da directory
    os.makedirs(airfoil_loc)
    shutil.copytree('backup', airfoil_loc+'1gen')
    pass

## file and data mining

In [3]:
from os import listdir


def foil_mining(airfoil_loc):
    #get only the files in a directory 
    #if boo=True, only get files, if boo=False
    file_list = [f for f in listdir(airfoil_loc)]
    # get only the airfoil name
    for f in file_list:
        yield f.split('.')[0]

def file_mining(file):
    #open file and convert them into list
    f = list(open(file, 'r'))
    #only select those data (need to be test if there is a buy)
    f = f[12:]
    #contains a list contains all the [alpha, CL]
    output = [i.split()[:2] for i in f]
    #since now output value elements are string, we need to con conver them to float
    output = [list(map(float, row)) for row in output]
    return output

def data_mining(data_loc,reys):
    #make the data directory of the airfoil
    airfoils = foil_mining(data_loc)
    for airfoil in airfoils:
        loc = data_loc+airfoil+'/'
        outputs = []
        #use foil mining function to mine reynolds number
        reys = foil_mining(loc)
        for rey in reys:
            #get the output for each reynolds number
            output = file_mining(loc+str(rey)+'.txt')
            #put rey data with angle value
            outputs.append([rey,output])
            #example: [rey, [*angle, *cl, *cd]]
        yield airfoil, outputs

# algorithm

### Fitness function idea
    1: reynolds number score:   rey: weights
                                0.5: 1
                                0.2: 2
    
    2: angle data score:        angle: weights
                                13: 2
                                
                                
    3: sum up those points
    
    
    20% thickness
    more importantly improve lift

In [4]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def data_check(cl):
    if max(cl) < 1.4:
        return True
    else:
        return False
    
def frange(start, finish, interval=1.0):
    #frange starts from small angle to large angle
    assert finish > start, 'frange start with smaller number'
    x = float(start)
    out = [x]
    while x < float(finish):
        x+=interval
        out.append(round(x, 5))
    return out

    give weighting to Cd
    fix flow condition
    loss function explaination

In [5]:
import numpy as np
def heuristic(mata_data):
    #contains: airfoil info, rey, angles, data
    #for each reynolds number
    final_score = {}
    for airfoil, data in mata_data:
        #data [angle, cl]
        score = 0
        for i in data:
            #get score for the each reynolds number
            angleP = heuristic_angle(i[1])
            #weight them by different reynolds number
            reyP = heuristic_rey(i[0])
            #sum them up
            score += angleP * reyP
            score = float('%.3f' % round(score, 3))
        final_score.update({airfoil: score})
    return final_score

def heuristic_rey(rey):
    #haven't decide the model yet!!!!!
    '''need to discuss this'''
    return 1

def heuristic_angle(datas):
    #set initial point
    final_point = 0
    if datas:
        data = np.transpose(datas)
        #use costumized sigmoid function to get weights
        ## mainly to enhance the importance of the Cls above 8 degree AOT
        weights = 10 * sigmoid((data[0]-3) / 2.5) + 0.01
        #get the score for each angle
        points = data[1] * weights
        #positive all the negative term
        points = np.sqrt(points**2)
        #averge them and times 10
        final_point = np.average(points) * 10
    else:
        pass
    #only output the score up to 3 decimal places
    return final_point

In [6]:
import matplotlib.pyplot as plt

x = np.array([n for n in frange(-10, 12, 0.1)])
weight = 10 * sigmoid((x-3) / 2.5) + 0.01
plt.plot(x, weight)
plt.show()

The sigmoid function is to evaluate airfoil performance in a particular fashion. Since we are dealing with subsonic wind turbine airfoil. The operational angle of attacks are usually around 10 to 13 degrees. therefore we are aiming to enhance the importance of the angle score above 10 degree, and slimmer the importance below that. 

AOA below 0 degree sometimes will have negative Cl, therefore I set the value as 0.01 to diminish the importance but not canceling the value

In [7]:
import operator
def sort_airfoil(data, old_champs=[]):
    #sort airfoil according to score from big to small
    candidates = list(data.items())
    candidates += old_champs
    return (sorted(candidates, key=operator.itemgetter(1), reverse=True))

### airfoil generator

In [8]:
def file2cor(files_loc):
    '''this is to get the airfoil cordinates in list'''
    airfoil_cor = {}
    for airfoil, file in files_loc.items():
        #open files
        f = list(open(file, 'r'))
        # get rid of \n term and space
        cordinate = [(i.split('\n')[0]).split(' ')[2:] for i in f]
        cordinate = [list(filter(lambda x: len(x) > 0, cor)) for cor in cordinate]
        cordinate = [list(map(float, row)) for row in cordinate]
        #therefore this becomes [[x1, y1], [x2, y2], [x3, y3] ...]
        
        ''' don't convert object'''
        #put the generator back to the dictionary
        airfoil_cor[airfoil] = cordinate
        
    return airfoil_cor

def cor2file(cordinates, loc=''):
    assert type(cordinates) == dict, 'dude, this is dictionary parameter'
    if not os.path.isdir(loc):
        os.makedirs(loc)
    for airfoil, plots in cordinates.items():
        #create a write file
        airfoil_loc = loc+airfoil+'.txt'
        f = open(airfoil_loc, 'w+')
        #turn every point into string format for joining
        plots = list(plots)
        plots_str = [list(map(str, row)) for row in (plots)]
        #turned them into right format for writing into files
        write_point = ['   '.join(row) for row in plots_str]
        #write each point's location into the file
        for row in write_point:
            f.write(row+'\n')
        f.close()
        #if thickness doesn't above 20%
        #if not thickness(airfoil_loc, 0.2):
        #    os.remove(airfoil_loc)
    pass

In [9]:
import random
import itertools as it

def generator(parents_loc, mutation_intensity, n_gen):
    #open those locations and get the plots
    parents = file2cor(parents_loc)
    #set children airfoil
    new_airfoil = {}
    #gen name
    surname = 'G'+str(n_gen).zfill(2)
    #airfoil name
    n_foil = 0
    first_name = 'A{}'.format
    
    #generate all the possible combinations
    couples = list(it.combinations(list(parents.keys()), 2))
    for father, mother in couples:
        #papa and mama produce some children
        try:
            child1, child2 = reproduction(parents[father], parents[mother], mutation_intensity)
        except:
            print(father, mother)

        #get airfoil serial number and put them into dictionary
        n_foil+=1
        new_airfoil[surname+first_name(n_foil)] = child1
                
        n_foil+=1
        new_airfoil[surname+first_name(n_foil)] = child2
        
    return new_airfoil

def reproduction(father, mother, mutation_intensity):
    #every airfoil should start and end with [1.0, 0.0]
    father = list(father)
    mother = list(mother)
    
    #find the split point
    if [0.0, 0.0] in father:
        idxF = father.index([0.0, 0.0])
    else:
        print(father)
    if [0.0, 0.0] in mother:
        idxM = mother.index([0.0, 0.0])
    else:
        print(mother)
    
    
    #split genes
    genesUp = [father[:idxF], mother[:idxF]]
    genesDown = [father[idxF+1:], mother[idxM+1:]]
    
    #little bit mutation
    genesUp = mutation(genesUp, mutation_intensity)
    genesDown = mutation(genesDown, mutation_intensity)
    
    #new airfoil
    child1 = genesUp[0] + [[0.0, 0.0]] + genesDown[1]
    child2 = genesUp[1] + [[0.0, 0.0]] + genesDown[0]
    
    #make sure they are in good formate fist and last are [1.0, 0.0]
    child1[0] = child1[-1] = [1.0, 0.0]
    child2[0] = child2[-1] = [1.0, 0.0]
    
    return child1, child2

def mutation(genes, Kmu):
    for gene in genes:
        for alleles in gene:
            if random.random() < Kmu:
                alleles[1] = float(alleles[1]) + random.uniform(-1.0, 1.0)/1000
    return genes

## script and setup

this is to setup the script and couple the python with Xfoil
    

__xfoil commend:__
    
    LOAD dat/e1211-il.dat	Load the dat file
    MDES	Go to the MDES menu
    FILT	Smooth any variations in the dat file data
    EXEC	Execute the smoothing
            Back to main menu
    PANE	Set the number and location of the airfoil points for analysis
    OPER	Go to the OPER menu
    ITER 70	Max number of iterations set to 70 for convergence
    RE 50000	Set Reynolds number (required?)
    VISC 50000	Set viscous calculation with Reynolds number
    PACC	Start polar output file
    polar/e1211-il_50000.txt	The output polar file name
            No dump file
    ALFA 0	Calculate lift and drag at 0° angle of attack
    ALFA 0.25	... 0.25°
    ALFA 0.5	... 0.5° ...
    ...	...more alpha calculations here ...
    ALFA 3.5	At 3.5° no convergence
    ALFA 3.5	... try again ...
    ALFA 3.5	... and again
    INIT	Run INIT to reinitialise
    ALFA 3.75	Skip to 3.75°
    ...	...rest of alpha calculations here ...
    PACC	Close polar file
    VISC	Reinitialise viscous calculation (required?)
    Down to main menu
    QUIT	Exit Xfoil

In [10]:
def script(airfoil_loc, airfoil, data_loc, AOT, RE):
    load_script = ['load '+airfoil_loc+airfoil+".txt", 
            airfoil,
            'MDES',
            'FILT',
            'EXEC',
            ' ',
            'PANE',
            'OPER',
            'ITER 150']       
    #define the output file name (different reynolds number has different file)
    data_file_name = data_loc+'/'+str(RE)+'.txt'
    #check and create file
    constant_set = ['RE ' + str(RE),
                    'VISC ' +str(RE),
                    'PACC', 
                    data_file_name,
                    ' ']
    #analyse different angle of attack
    alpha = ['ALFA '+str(a) for a in AOT]
    init = constant_set + alpha + ['PACC', ' ',]
    load_script += init
    load_script += [' ', ' ', 'QUIT\n']
    return load_script

In [11]:
def execute(airfoil_loc, airfoil, data_loc, angles, rey_range):
    #create a directory for each airfoil
    filedir = data_loc+airfoil
    if not os.path.isdir(filedir):
        os.makedirs(filedir)
    for RE in rey_range:
        #coupling xfoil into python
        if not isfile(filedir+'/'+str(RE)+'.txt'):
            ps = sp.Popen(['xfoil.exe'],
                          stdin=sp.PIPE,
                          stdout=None,
                          stderr=None,
                          encoding='utf8')
            #execute the script
            res = ps.communicate('\n'.join(script(airfoil_loc,airfoil, filedir, angle_range, RE)))
   

In [12]:
def thickness(airfoil_loc, min_thick):
    '''this is to check if the airfoil have sufficient thickness'''
    boo = False
    
    ps = sp.Popen(['xfoil.exe'],    
                  stdin=sp.PIPE,                  
                  stdout=sp.PIPE,                  
                  stderr=None,                  
                  encoding='utf8')       
    #execute the script   
    res = ps.communicate('\n'.join(['load '+airfoil_loc, 'airfoil\n']))
    
    #get the thickness index of the airfoil
    result_text = list(filter(None, str(res[0]).split(' ')))
    thickness_loc = result_text.index('thickness') + 2
    thickness = float(result_text[thickness_loc])
    print(thickness)
    if thickness > min_thick:
        boo = True
    return boo

## run

This is to execute the whole programme with genetic algorithm.
    
    variables:
    epochs
    airfoil
    data_loc
    airfoil_loc
    reynolds number range
    angle range

In [14]:
#constants setting
epochs = 2
airfoil_dir = 'airfoil/'
data_dir = 'data/'
'why do we choose this angle of attack'
angle_range = frange(-10, 10, 0.1)
'how to get this value?'
rey_range = range(200000,400000, 100000)
n_winners = 3
mutation_intensity = 0.2

In [15]:
#reboost the system everytime it starts
boostup(data_dir, airfoil_dir)


In [16]:
from tqdm import tqdm_notebook

# best airfoils that is going to produce children
winners = []
# those airfoils file location
winners_loc = {}
for n in tqdm_notebook(range(epochs), desc='progress'):
    #set generation
    gen = str(n+1)
    #generate locations
    data_loc = data_dir+gen+'gen/'
    airfoil_loc = (airfoil_dir+'{}'+'gen/').format
    
    #execute airfoil to each of the airfoil in the folder
    airfoil_list = foil_mining(airfoil_loc(gen))
    for airfoil in tqdm_notebook(airfoil_list, desc='airfoil simulations', leave=False):
        print(airfoil)
        ang_range = list(angle_range)
        execute(airfoil_loc(gen), airfoil, data_loc, ang_range, rey_range)
    
    #get lift coefficient of each angle in each reynolds number
    mata_data = data_mining(data_loc, rey_range)
    
    # basically, the winners will be reused in the next generation for comparation see if the next generation got better
    #get the scores in such order than the highers one is the first one
    '''we can see, here, the last gen winner will be comparing with this gen data as well'''
    scores = sort_airfoil(heuristic(mata_data), winners)

    # choose the number of parents in this model
    winners = scores[:n_winners]
    #get the location in dict formate
    winners_loc.update({i: airfoil_loc(gen)+i+'.txt' for i, _ in winners if i not in winners_loc})
    winners_loc = {airfoil: winners_loc[airfoil] for airfoil, _ in winners}
    
    print(winners)
    print(winners_loc)
    
    #design new airfoil usint genetic algorithm
    new_airfoil = generator(winners_loc, mutation_intensity, n+1)
    #new generation for saving new airfoil
    gen = str(n+2)
    #save new airfoil
    cor2file(new_airfoil, airfoil_loc(gen))

HBox(children=(IntProgress(value=0, description='progress', max=2), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', description='airfoil simulations', max=1), HTML(value='')))

NACA4421
S813



KeyboardInterrupt: 

In [ ]:
li = [1,2,3,4,5]
li.index(0) == li.index(0)

In [ ]:
airfoil_loc = (airfoil_dir+'{}'+'gen/').format
airfoil_loc(3)

In [ ]:
string(airfoil_loc)

In [ ]:
#set generation
gen = 'gen'
#generate locations
data_loc = data_dir+gen+'/'
airfoil_loc = airfoil_dir+gen+'/'
# best airfoils that is going to produce children
winners = []
# those airfoils file location
winners_loc = {}

In [ ]:
mata_data = data_mining(data_loc, rey_range)

scores = sort_airfoil(heuristic(mata_data), winners)

# basically, the winners will be reused in the next generation for comparation see if the next generation got better
# choose the number of parents in this model
winners = scores[:num_parents]
#get the location in dict formate
winners_loc.update({i: airfoil_loc+i+'.txt' for i, _ in winners if i not in winners_loc})
winners_loc = {airfoil: winners_loc[airfoil] for airfoil, _ in winners}

In [ ]:
winners